In [4]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

import pandas as pd
pd.options.display.max_columns = 100

from matplotlib import pyplot as plt
import numpy as np

import seaborn as sns
sns.set(rc={'figure.figsize':(12,9)})
import pylab as plot
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss,accuracy_score
import pickle
import time

In [5]:
from xgboost import XGBClassifier
import xgboost as xgb

In [6]:
#for scaling
from sklearn.preprocessing import StandardScaler

Importing training dataset

In [7]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [8]:
print(data.shape)

(867873, 10)


In [9]:
data.columns

Index(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y', 'Id'],
      dtype='object')


Splitting the "Dates" into date and time

In [10]:
df=data.head(100000)
datetime=df.Dates.str.split(pat=" ",expand=True)
datetime.columns=['Date','Time']
#datetime

#------------test data-------------

df_test=test
datetime_test=df_test.Dates.str.split(pat=" ",expand=True)
datetime_test.columns=['Date','Time']

In [11]:
Date=datetime.Date.str.split(pat="-",expand=True)
Date.columns=['Year','Month','Day']

Time=datetime.Time.str.split(pat=":",expand=True)
Time.columns=['Hour','Minute','Second']

#------------test data-------------

Date_test=datetime_test.Date.str.split(pat="-",expand=True)
Date_test.columns=['Year','Month','Day']

Time_test=datetime_test.Time.str.split(pat=":",expand=True)
Time_test.columns=['Hour','Minute','Second']

Dropping the column of "Dates" and appending the dataframe "datetime"

In [12]:
df=pd.concat([df,Date,Time],axis=1)
#df

#-----------test data------------------

df_test=pd.concat([df_test,Date_test,Time_test],axis=1)


In [13]:
df=df.drop(labels=['Dates'],axis=1)

#-----------test data------------------

df_test=df_test.drop(labels=['Dates'],axis=1)


# #Encoding

In [14]:
df.columns

Index(['Category', 'Descript', 'DayOfWeek', 'PdDistrict', 'Resolution',
       'Address', 'X', 'Y', 'Id', 'Year', 'Month', 'Day', 'Hour', 'Minute',
       'Second'],
      dtype='object')

LabelEncoding of "DayOfWeek" and "PdDistrict"

In [15]:
le = preprocessing.LabelEncoder()

In [16]:
#y=pd.get_dummies(df.Category,columns=['Category'],prefix=" ",prefix_sep=" ",drop_first=True,)

le_res=le.fit_transform(df['Category'])
y=pd.DataFrame(le_res)
y.columns=['Category']
#y

In [17]:
y['Category'].nunique()

36

# Rotation 60 degree

In [18]:
df["rot60_X"]=(0.5) * df["Y"] + (1.732/2) * df["X"]
df["rot60_Y"]=0.5 * df["Y"] - (1.732/2) * df["X"]



df_test["rot60_X"]=(0.5) * df_test["Y"] + (1.732/2) * df_test["X"]
df_test["rot60_Y"]=0.5 * df_test["Y"] - (1.732/2) * df_test["X"]

In [19]:
df["radial60"]=np.sqrt(np.power(df['rot60_X'],2) + np.power(df['rot60_Y'],2))

df_test["radial60"]=np.sqrt(np.power(df_test['rot60_X'],2) + np.power(df_test['rot60_Y'],2))

# Month clubbing

In [20]:
df=df.drop(labels='rot60_X',axis=1)

df_test=df_test.drop(labels='rot60_X',axis=1)

In [21]:
df=df.drop(labels='rot60_Y',axis=1)

df_test=df_test.drop(labels='rot60_Y',axis=1)

In [22]:
df=df.drop(labels='Second',axis=1)

df_test=df_test.drop(labels='Second',axis=1)

# Minute Clubbing

In [23]:
df['Minute']=df['Minute'].apply(lambda x:int(x))
df['Minute']=df['Minute'].apply(lambda x : 'low' if x <31 else 'high')

df_test['Minute']=df_test['Minute'].apply(lambda x:int(x))
df_test['Minute']=df_test['Minute'].apply(lambda x : 'low' if x <31 else 'high')


# DayOfWeek

In [24]:
df['DayOfWeek']= df['DayOfWeek'].apply(lambda x : 'WeekHigh' if x in ('Wednesday','Friday') else ('WeekMed' if x in ('Tuesday','Thursday','Saturday') else 'WeekLow'))


df_test['DayOfWeek']= df_test['DayOfWeek'].apply(lambda x : 'WeekHigh' if x in ('Wednesday','Friday') else ('WeekMed' if x in ('Tuesday','Thursday','Saturday') else 'WeekLow'))


# Address Update

In [25]:
df['Intersection']=df['Address'].apply(lambda x : 1 if '/' in x else 0)
df['Block']=df['Address'].apply(lambda x : 1 if 'Block' in x else 0)
df_test['Intersection']=df_test['Address'].apply(lambda x : 1 if '/' in x else 0)
df_test['Block']=df_test['Address'].apply(lambda x : 1 if 'Block' in x else 0)

In [26]:
address=pd.DataFrame(df['Address'],columns=['Address'])
address=address.Address.str.split(pat=" /",expand=True )

address.columns=['Address','Intr2']

address=address.Address.str.split(pat=" /",expand=True )
address.columns=['Address']

string=address.iloc[:,0]
string=string.str.strip()

address_fram=string.to_frame()
temp=address_fram['Address'].astype(str).str[-2:]

address=temp.to_frame()

address['Address']=address['Address'].apply(lambda x :( x if x in ("ST","AV","LN","DR","BL","HY","CT","RD","PL","PZ","TR","AL","CR","WK","EX","RW") else (("I-80" if x in ("80") else ("HWY" if x in ("WY") else ("WAY" if x in ("AY") else ("TER" if x in ("ER") else ("ALMS" if x in ("MS") else ("MAR" if x in ("AR") else ("PARK" if x in ("RK") else ("STWY" if x in ("WY") else ("VIA" if x in ("NO") else ("BLOCK")))))))))))))
df=df.drop(labels=['Address'],axis=1)
df=pd.concat([address,df],axis=1)



# Address splitting

In [27]:
address_test=pd.DataFrame(df_test['Address'],columns=['Address'])
address_test=address_test.Address.str.split(pat=" /",expand=True )

address_test.columns=['Address','Intr2']

address_test=address_test.Address.str.split(pat=" /",expand=True )
address_test.columns=['Address']

string_test=address_test.iloc[:,0]
string_test=string_test.str.strip()

address_fram_test=string_test.to_frame()
temp_test=address_fram_test['Address'].astype(str).str[-2:]

address_test=temp_test.to_frame()

address_test['Address']=address_test['Address'].apply(lambda x :( x if x in ("ST","AV","LN","DR","BL","HY","CT","RD","PL","PZ","TR","AL","CR","WK","EX","RW") else (("I-80" if x in ("80") else ("HWY" if x in ("WY") else ("WAY" if x in ("AY") else ("TER" if x in ("ER") else ("ALMS" if x in ("MS") else ("MAR" if x in ("AR") else ("PARK" if x in ("RK") else ("STWY" if x in ("WY") else ("VIA" if x in ("NO") else ("BLOCK")))))))))))))
df_test=df_test.drop(labels=['Address'],axis=1)
df_test=pd.concat([address_test,df_test],axis=1)

In [28]:
Id=df['Id']
df=df.drop(['Descript','Resolution','Id'],axis=1)

#----------test data---------

Id_test=df_test['Id']
df_test=df_test.drop(['Descript','Resolution','Id'],axis=1)

In [29]:
df_test.columns

Index(['Address', 'DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'Day',
       'Hour', 'Minute', 'radial60', 'Intersection', 'Block'],
      dtype='object')

In [30]:
data['PdDistrict'].unique()

array(['MISSION', 'SOUTHERN', 'BAYVIEW', 'CENTRAL', 'INGLESIDE',
       'NORTHERN', 'RICHMOND', 'TARAVAL', 'TENDERLOIN', 'PARK'],
      dtype=object)

# Label Encoding

In [31]:

df.columns

Index(['Address', 'Category', 'DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year',
       'Month', 'Day', 'Hour', 'Minute', 'radial60', 'Intersection', 'Block'],
      dtype='object')

In [32]:
df_test.columns

Index(['Address', 'DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'Day',
       'Hour', 'Minute', 'radial60', 'Intersection', 'Block'],
      dtype='object')

In [33]:
le = preprocessing.LabelEncoder()

Label Encoding :DayOfWeek

In [34]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['DayOfWeek'])
Day=pd.DataFrame(le_res)
Day.columns=['DayOfWeek']
df=df.drop(labels=['DayOfWeek'],axis=1)
df=pd.concat([Day,df],axis=1)

#----------test data----------

le_res_test=le.fit_transform(df_test['DayOfWeek'])
Day_test=pd.DataFrame(le_res_test)
Day_test.columns=['DayOfWeek']
df_test=df_test.drop(labels=['DayOfWeek'],axis=1)
df_test=pd.concat([Day_test,df_test],axis=1)

Label Encoding: PdDistrict

In [35]:

le_res=le.fit_transform(df['PdDistrict'])
District=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
District.columns=['District']
df=df.drop(labels=['PdDistrict'],axis=1)
df=pd.concat([District,df],axis=1)


le_res_test=le.fit_transform(df_test['PdDistrict'])
District_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
District_test.columns=['District']
df_test=df_test.drop(labels=['PdDistrict'],axis=1)
df_test=pd.concat([District_test,df_test],axis=1)


Label encoding: Year

In [36]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Year'])
Year=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Year.columns=['Year']
df=df.drop(labels=['Year'],axis=1)
df=pd.concat([Year,df],axis=1)


#le = preprocessing.LabelEncoder()
le_res_test=le.fit_transform(df_test['Year'])
Year_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Year_test.columns=['Year']
df_test=df_test.drop(labels=['Year'],axis=1)
df_test=pd.concat([Year_test,df_test],axis=1)


Label encoding: Month

In [37]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Month'])
Month=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Month.columns=['Month']
df=df.drop(labels=['Month'],axis=1)
df=pd.concat([Month,df],axis=1)


#le = preprocessing.LabelEncoder()
le_res_test=le.fit_transform(df_test['Month'])
Month_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Month_test.columns=['Month']
df_test=df_test.drop(labels=['Month'],axis=1)
df_test=pd.concat([Month_test,df_test],axis=1)


Label encoding: Day

In [38]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Day'])
Day=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Day.columns=['Day']
df=df.drop(labels=['Day'],axis=1)
df=pd.concat([Day,df],axis=1)


#le = preprocessing.LabelEncoder()
le_res_test=le.fit_transform(df_test['Day'])
Day_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Day_test.columns=['Day']
df_test=df_test.drop(labels=['Day'],axis=1)
df_test=pd.concat([Day_test,df_test],axis=1)


Label encoding: Hour

In [39]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Hour'])
Hour=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Hour.columns=['Hour']
df=df.drop(labels=['Hour'],axis=1)
df=pd.concat([Hour,df],axis=1)


#le = preprocessing.LabelEncoder()
le_res_test=le.fit_transform(df_test['Hour'])
Hour_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Hour_test.columns=['Hour']
df_test=df_test.drop(labels=['Hour'],axis=1)
df_test=pd.concat([Hour_test,df_test],axis=1)


Label encoding: Minute

In [40]:

le_res=le.fit_transform(df['Minute'])
Minute=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Minute.columns=['Minute']
df=df.drop(labels=['Minute'],axis=1)
df=pd.concat([Minute,df],axis=1)


le_res_test=le.fit_transform(df_test['Minute'])
Minute_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Minute_test.columns=['Minute']
df_test=df_test.drop(labels=['Minute'],axis=1)
df_test=pd.concat([Minute_test,df_test],axis=1)



#label encode street suffix

In [41]:
le_res=le.fit_transform(df['Category'])
cat=pd.DataFrame(le_res)
cat.columns=['Category']
df=df.drop(labels=['Category'],axis=1)
df=pd.concat([cat,df],axis=1)

df.columns

Index(['Category', 'Minute', 'Hour', 'Day', 'Month', 'Year', 'District',
       'DayOfWeek', 'Address', 'X', 'Y', 'radial60', 'Intersection', 'Block'],
      dtype='object')

In [42]:

le_res=le.fit_transform(df['Address'])
Address=pd.DataFrame(le_res)
Address.columns=['Address']
df=df.drop(labels=['Address'],axis=1)
df=pd.concat([Address,df],axis=1)
le_res=le.fit_transform(df_test['Address'])
Address_test=pd.DataFrame(le_res)
Address_test.columns=['Address']
df_test=df_test.drop(labels=['Address'],axis=1)
df_test=pd.concat([Address_test,df_test],axis=1)

# PCA

In [43]:
from sklearn.decomposition import PCA
dimension=np.array(np.array(df.iloc[:,9:11]))
pca = PCA(n_components=1)
dim=pca.fit_transform(dimension)
dim=pd.DataFrame(dim)
dim.columns=['X1']
df=df.drop(labels=['X','Y'],axis=1)
df=pd.concat([dim,df],axis=1)



In [44]:
dimension_test=np.array(np.array(df_test.iloc[:,9:11]))
pca= PCA(n_components=1)
dim_test=pca.fit_transform(dimension_test)
dim_test=pd.DataFrame(dim_test)
dim_test.columns=['X1']
df_test=df_test.drop(labels=['X','Y'],axis=1)
df_test=pd.concat([dim_test,df_test],axis=1)

In [45]:
df=df[['X1', 'Year','Hour','Minute','Address', 'Intersection','Block']]


df_test=df_test[[ 'X1', 'Year','Hour','Minute','Address', 'Intersection','Block']]
#'radial60'

In [46]:
A=df.corr()
A

,X1,Year,Hour,Minute,Address,Intersection,Block
X1,1.000000,-0.006490,-0.002111,0.003670,0.021928,0.014627,-0.014627
Year,-0.006490,1.000000,-0.009618,-0.015420,0.009450,-0.026512,0.026512
Hour,-0.002111,-0.009618,1.000000,0.003234,0.003987,0.048190,-0.048190
Minute,0.003670,-0.015420,0.003234,1.000000,-0.022630,-0.040247,0.040247
Address,0.021928,0.009450,0.003987,-0.022630,1.000000,0.053993,-0.053993
Intersection,0.014627,-0.026512,0.048190,-0.040247,0.053993,1.000000,-1.000000
Block,-0.014627,0.026512,-0.048190,0.040247,-0.053993,-1.000000,1.000000


# Splitting into test and train


In [47]:
X=df

In [48]:
#splitting the dataset into the training set and test set
#from sklearn import model_selection as ms

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4,shuffle=False,random_state = 1023)

In [49]:
X_test.head()

,X1,Year,Hour,Minute,Address,Intersection,Block
67000,-0.004018,10,11,1,2,0,1
67001,-0.003859,9,17,1,2,0,1
67002,-0.031106,7,2,1,2,0,1
67003,-0.036559,8,18,1,17,1,0
67004,0.006435,9,12,1,17,0,1


# Individual Model

In [50]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [61]:
model11= xgb.XGBClassifier(objective='multi:softprob',learning_rate =0.2,max_depth=4,min_child_weight=3)

In [ ]:
model11= xgb.XGBClassifier(objective='multi:softprob')
model11.fit(X_train, y_train)
y_final=model11.predict(X_test)
print (log_loss(y_test,y_final));

C:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [47]:
import lightgbm as lgb
model12= lgb.LGBMClassifier(objective='multiclass')
model12.fit(X_train,y_train)
y_final=model12.predict_proba(X_test)
print (log_loss(y_test,y_final));

C:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


2.448049808290997


In [49]:
from catboost import CatBoostClassifier
model13= CatBoostClassifier(loss_function="MultiClass")
model13.fit(X_train,y_train,use_best_model=True)
y_final=model13.predict_proba(X_test)
print (log_loss(y_test,y_final));

You should provide test set for use best model. use_best_model parameter swiched to false value.


0:	learn: -3.4936530	total: 321ms	remaining: 5m 20s
1:	learn: -3.4205474	total: 645ms	remaining: 5m 21s
2:	learn: -3.3579533	total: 977ms	remaining: 5m 24s
3:	learn: -3.3035574	total: 1.3s	remaining: 5m 24s
4:	learn: -3.2549589	total: 1.63s	remaining: 5m 23s
5:	learn: -3.2102096	total: 1.96s	remaining: 5m 24s
6:	learn: -3.1709330	total: 2.28s	remaining: 5m 23s
7:	learn: -3.1338873	total: 2.6s	remaining: 5m 22s
8:	learn: -3.1000195	total: 2.92s	remaining: 5m 21s
9:	learn: -3.0695024	total: 3.24s	remaining: 5m 20s
10:	learn: -3.0414257	total: 3.56s	remaining: 5m 19s
11:	learn: -3.0151838	total: 3.88s	remaining: 5m 19s
12:	learn: -2.9904306	total: 4.2s	remaining: 5m 19s
13:	learn: -2.9675612	total: 4.52s	remaining: 5m 18s
14:	learn: -2.9459954	total: 4.84s	remaining: 5m 18s
15:	learn: -2.9258577	total: 5.17s	remaining: 5m 17s
16:	learn: -2.9069630	total: 5.48s	remaining: 5m 17s
17:	learn: -2.8889847	total: 5.8s	remaining: 5m 16s
18:	learn: -2.8718205	total: 6.11s	remaining: 5m 15s
19:	lea

155:	learn: -2.4685393	total: 48s	remaining: 4m 19s
156:	learn: -2.4681418	total: 48.3s	remaining: 4m 19s
157:	learn: -2.4677310	total: 48.6s	remaining: 4m 18s
158:	learn: -2.4673370	total: 48.9s	remaining: 4m 18s
159:	learn: -2.4669099	total: 49.2s	remaining: 4m 18s
160:	learn: -2.4664693	total: 49.5s	remaining: 4m 17s
161:	learn: -2.4661022	total: 49.8s	remaining: 4m 17s
162:	learn: -2.4656801	total: 50s	remaining: 4m 16s
163:	learn: -2.4653425	total: 50.3s	remaining: 4m 16s
164:	learn: -2.4648870	total: 50.6s	remaining: 4m 16s
165:	learn: -2.4644182	total: 51s	remaining: 4m 16s
166:	learn: -2.4640417	total: 51.3s	remaining: 4m 15s
167:	learn: -2.4637575	total: 51.5s	remaining: 4m 15s
168:	learn: -2.4633323	total: 51.8s	remaining: 4m 14s
169:	learn: -2.4630384	total: 52.1s	remaining: 4m 14s
170:	learn: -2.4627363	total: 52.4s	remaining: 4m 14s
171:	learn: -2.4623695	total: 52.7s	remaining: 4m 13s
172:	learn: -2.4620008	total: 53s	remaining: 4m 13s
173:	learn: -2.4617914	total: 53.3s	

307:	learn: -2.4285536	total: 1m 35s	remaining: 3m 33s
308:	learn: -2.4283800	total: 1m 35s	remaining: 3m 33s
309:	learn: -2.4281740	total: 1m 35s	remaining: 3m 32s
310:	learn: -2.4279266	total: 1m 35s	remaining: 3m 32s
311:	learn: -2.4276743	total: 1m 36s	remaining: 3m 32s
312:	learn: -2.4274958	total: 1m 36s	remaining: 3m 31s
313:	learn: -2.4272657	total: 1m 36s	remaining: 3m 31s
314:	learn: -2.4270814	total: 1m 37s	remaining: 3m 31s
315:	learn: -2.4269546	total: 1m 37s	remaining: 3m 30s
316:	learn: -2.4267944	total: 1m 37s	remaining: 3m 30s
317:	learn: -2.4266104	total: 1m 37s	remaining: 3m 30s
318:	learn: -2.4263605	total: 1m 38s	remaining: 3m 29s
319:	learn: -2.4261821	total: 1m 38s	remaining: 3m 29s
320:	learn: -2.4259949	total: 1m 38s	remaining: 3m 29s
321:	learn: -2.4257657	total: 1m 39s	remaining: 3m 28s
322:	learn: -2.4255903	total: 1m 39s	remaining: 3m 28s
323:	learn: -2.4254158	total: 1m 39s	remaining: 3m 28s
324:	learn: -2.4251760	total: 1m 39s	remaining: 3m 27s
325:	learn

458:	learn: -2.4029862	total: 2m 18s	remaining: 2m 43s
459:	learn: -2.4028279	total: 2m 18s	remaining: 2m 42s
460:	learn: -2.4027207	total: 2m 19s	remaining: 2m 42s
461:	learn: -2.4025345	total: 2m 19s	remaining: 2m 42s
462:	learn: -2.4022972	total: 2m 19s	remaining: 2m 41s
463:	learn: -2.4021151	total: 2m 19s	remaining: 2m 41s
464:	learn: -2.4019896	total: 2m 20s	remaining: 2m 41s
465:	learn: -2.4018646	total: 2m 20s	remaining: 2m 41s
466:	learn: -2.4017442	total: 2m 20s	remaining: 2m 40s
467:	learn: -2.4015892	total: 2m 21s	remaining: 2m 40s
468:	learn: -2.4014607	total: 2m 21s	remaining: 2m 40s
469:	learn: -2.4013453	total: 2m 21s	remaining: 2m 39s
470:	learn: -2.4012259	total: 2m 21s	remaining: 2m 39s
471:	learn: -2.4010590	total: 2m 22s	remaining: 2m 39s
472:	learn: -2.4008940	total: 2m 22s	remaining: 2m 38s
473:	learn: -2.4006877	total: 2m 22s	remaining: 2m 38s
474:	learn: -2.4005327	total: 2m 23s	remaining: 2m 38s
475:	learn: -2.4004218	total: 2m 23s	remaining: 2m 37s
476:	learn

609:	learn: -2.3808183	total: 3m 2s	remaining: 1m 56s
610:	learn: -2.3807500	total: 3m 2s	remaining: 1m 56s
611:	learn: -2.3805819	total: 3m 2s	remaining: 1m 55s
612:	learn: -2.3804446	total: 3m 2s	remaining: 1m 55s
613:	learn: -2.3803669	total: 3m 3s	remaining: 1m 55s
614:	learn: -2.3802578	total: 3m 3s	remaining: 1m 54s
615:	learn: -2.3801350	total: 3m 3s	remaining: 1m 54s
616:	learn: -2.3800244	total: 3m 4s	remaining: 1m 54s
617:	learn: -2.3798928	total: 3m 4s	remaining: 1m 53s
618:	learn: -2.3797243	total: 3m 4s	remaining: 1m 53s
619:	learn: -2.3795662	total: 3m 4s	remaining: 1m 53s
620:	learn: -2.3794816	total: 3m 5s	remaining: 1m 53s
621:	learn: -2.3793627	total: 3m 5s	remaining: 1m 52s
622:	learn: -2.3792622	total: 3m 5s	remaining: 1m 52s
623:	learn: -2.3791023	total: 3m 6s	remaining: 1m 52s
624:	learn: -2.3789944	total: 3m 6s	remaining: 1m 51s
625:	learn: -2.3788482	total: 3m 6s	remaining: 1m 51s
626:	learn: -2.3787135	total: 3m 6s	remaining: 1m 51s
627:	learn: -2.3786041	total

759:	learn: -2.3621306	total: 3m 45s	remaining: 1m 11s
760:	learn: -2.3620028	total: 3m 45s	remaining: 1m 10s
761:	learn: -2.3618891	total: 3m 45s	remaining: 1m 10s
762:	learn: -2.3617630	total: 3m 46s	remaining: 1m 10s
763:	learn: -2.3616430	total: 3m 46s	remaining: 1m 10s
764:	learn: -2.3615126	total: 3m 46s	remaining: 1m 9s
765:	learn: -2.3614115	total: 3m 47s	remaining: 1m 9s
766:	learn: -2.3613060	total: 3m 47s	remaining: 1m 9s
767:	learn: -2.3612188	total: 3m 47s	remaining: 1m 8s
768:	learn: -2.3610633	total: 3m 48s	remaining: 1m 8s
769:	learn: -2.3609624	total: 3m 48s	remaining: 1m 8s
770:	learn: -2.3608543	total: 3m 48s	remaining: 1m 7s
771:	learn: -2.3607133	total: 3m 48s	remaining: 1m 7s
772:	learn: -2.3605852	total: 3m 49s	remaining: 1m 7s
773:	learn: -2.3604678	total: 3m 49s	remaining: 1m 7s
774:	learn: -2.3603990	total: 3m 49s	remaining: 1m 6s
775:	learn: -2.3603021	total: 3m 50s	remaining: 1m 6s
776:	learn: -2.3602012	total: 3m 50s	remaining: 1m 6s
777:	learn: -2.3601098	

913:	learn: -2.3456222	total: 4m 29s	remaining: 25.4s
914:	learn: -2.3455274	total: 4m 30s	remaining: 25.1s
915:	learn: -2.3453773	total: 4m 30s	remaining: 24.8s
916:	learn: -2.3453123	total: 4m 30s	remaining: 24.5s
917:	learn: -2.3452161	total: 4m 31s	remaining: 24.2s
918:	learn: -2.3450862	total: 4m 31s	remaining: 23.9s
919:	learn: -2.3449811	total: 4m 31s	remaining: 23.6s
920:	learn: -2.3448458	total: 4m 31s	remaining: 23.3s
921:	learn: -2.3447444	total: 4m 32s	remaining: 23s
922:	learn: -2.3446431	total: 4m 32s	remaining: 22.7s
923:	learn: -2.3445074	total: 4m 32s	remaining: 22.4s
924:	learn: -2.3443676	total: 4m 33s	remaining: 22.1s
925:	learn: -2.3442576	total: 4m 33s	remaining: 21.8s
926:	learn: -2.3441521	total: 4m 33s	remaining: 21.5s
927:	learn: -2.3440469	total: 4m 33s	remaining: 21.2s
928:	learn: -2.3439970	total: 4m 34s	remaining: 21s
929:	learn: -2.3438709	total: 4m 34s	remaining: 20.7s
930:	learn: -2.3437637	total: 4m 34s	remaining: 20.4s
931:	learn: -2.3436614	total: 4m

In [63]:

gamma = [0.001]
param_grid = dict(gamma=gamma)
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=2)
grid_search = GridSearchCV(model11, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

C:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best: -2.460639 using {'gamma': 0.001}
-2.460639 (0.006378) with: {'gamma': 0.001}


# BLENDING

In [2]:
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.20,shuffle=False)

NameError: name 'train_test_split' is not defined

# Model-->1 XGBoost 

In [3]:
model1= xgb.XGBClassifier(objective='multi:softprob')
model1.fit(X_train, y_train)
val_pred1=model1.predict(X_val)
test_pred1=model1.predict(X_test)
val_pred1=pd.DataFrame(val_pred1)
test_pred1=pd.DataFrame(test_pred1)
val_pred1.columns=['Cat1']
test_pred1.columns=['Cat1']
#print (log_loss(y_test,y_pred1));

NameError: name 'xgb' is not defined

# Model-->2 LightGBM

In [52]:
import lightgbm as lgb
model2= lgb.LGBMClassifier(objective='multiclass')
model2.fit(X_train, y_train)
val_pred2=model2.predict(X_val)
test_pred2=model2.predict(X_test)
val_pred2=pd.DataFrame(val_pred2)
test_pred2=pd.DataFrame(test_pred2)
val_pred2.columns=['Cat2']
test_pred2.columns=['Cat2']

C:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [53]:
df_val=np.concatenate((np.array(X_val),np.array(val_pred1),np.array(val_pred2)),axis=1)
df_test=np.concatenate((np.array(X_test),np.array(test_pred1),np.array(test_pred2)),axis=1)

In [54]:
df_val=pd.DataFrame(df_val)
df_test=pd.DataFrame(df_test)

In [56]:
df_val.columns=['X1', 'Year','Hour','Minute','Address', 'Intersection','Block','Category1','Category2']
df_test.columns=['X1', 'Year','Hour','Minute','Address', 'Intersection','Block','Category1','Category2']

# Model-->3 CatBoost

In [57]:
from catboost import CatBoostClassifier
model3= CatBoostClassifier(loss_function="MultiClass")
model3.fit(df_val,y_val)
y_final=model3.predict_proba(df_test)
print (log_loss(y_test,y_final));

0:	learn: -3.4744658	total: 564ms	remaining: 9m 22s
1:	learn: -3.4086480	total: 1.2s	remaining: 9m 59s
2:	learn: -3.3521439	total: 1.9s	remaining: 10m 30s
3:	learn: -3.2995540	total: 2.63s	remaining: 10m 54s
4:	learn: -3.2531877	total: 3.35s	remaining: 11m 6s
5:	learn: -3.2117794	total: 4.02s	remaining: 11m 5s
6:	learn: -3.1749636	total: 4.56s	remaining: 10m 46s
7:	learn: -3.1410281	total: 5.1s	remaining: 10m 32s
8:	learn: -3.1095863	total: 5.81s	remaining: 10m 40s
9:	learn: -3.0806252	total: 6.35s	remaining: 10m 28s
10:	learn: -3.0535823	total: 6.96s	remaining: 10m 25s
11:	learn: -3.0279969	total: 7.53s	remaining: 10m 20s
12:	learn: -3.0036276	total: 8.29s	remaining: 10m 29s
13:	learn: -2.9815613	total: 9.07s	remaining: 10m 39s
14:	learn: -2.9606505	total: 9.92s	remaining: 10m 51s
15:	learn: -2.9408844	total: 10.6s	remaining: 10m 52s
16:	learn: -2.9223704	total: 11.4s	remaining: 10m 57s
17:	learn: -2.9052309	total: 12s	remaining: 10m 53s
18:	learn: -2.8884784	total: 12.7s	remaining: 1

153:	learn: -2.4705787	total: 1m 38s	remaining: 9m 2s
154:	learn: -2.4696678	total: 1m 39s	remaining: 9m 1s
155:	learn: -2.4694581	total: 1m 40s	remaining: 9m 1s
156:	learn: -2.4685279	total: 1m 40s	remaining: 9m
157:	learn: -2.4681705	total: 1m 41s	remaining: 8m 59s
158:	learn: -2.4673826	total: 1m 41s	remaining: 8m 59s
159:	learn: -2.4663957	total: 1m 42s	remaining: 8m 59s
160:	learn: -2.4652620	total: 1m 43s	remaining: 8m 58s
161:	learn: -2.4647968	total: 1m 43s	remaining: 8m 57s
162:	learn: -2.4639896	total: 1m 44s	remaining: 8m 57s
163:	learn: -2.4634643	total: 1m 45s	remaining: 8m 56s
164:	learn: -2.4628605	total: 1m 45s	remaining: 8m 56s
165:	learn: -2.4623531	total: 1m 46s	remaining: 8m 55s
166:	learn: -2.4617800	total: 1m 47s	remaining: 8m 54s
167:	learn: -2.4614440	total: 1m 47s	remaining: 8m 54s
168:	learn: -2.4613094	total: 1m 48s	remaining: 8m 52s
169:	learn: -2.4607434	total: 1m 49s	remaining: 8m 52s
170:	learn: -2.4600154	total: 1m 49s	remaining: 8m 51s
171:	learn: -2.45

304:	learn: -2.4072947	total: 3m 18s	remaining: 7m 31s
305:	learn: -2.4072110	total: 3m 18s	remaining: 7m 30s
306:	learn: -2.4067942	total: 3m 19s	remaining: 7m 30s
307:	learn: -2.4064748	total: 3m 20s	remaining: 7m 29s
308:	learn: -2.4064118	total: 3m 20s	remaining: 7m 28s
309:	learn: -2.4062908	total: 3m 21s	remaining: 7m 27s
310:	learn: -2.4059366	total: 3m 21s	remaining: 7m 27s
311:	learn: -2.4057557	total: 3m 22s	remaining: 7m 26s
312:	learn: -2.4054784	total: 3m 23s	remaining: 7m 25s
313:	learn: -2.4051401	total: 3m 23s	remaining: 7m 25s
314:	learn: -2.4047782	total: 3m 24s	remaining: 7m 24s
315:	learn: -2.4045311	total: 3m 25s	remaining: 7m 23s
316:	learn: -2.4041392	total: 3m 25s	remaining: 7m 23s
317:	learn: -2.4039381	total: 3m 26s	remaining: 7m 22s
318:	learn: -2.4033666	total: 3m 27s	remaining: 7m 22s
319:	learn: -2.4032806	total: 3m 27s	remaining: 7m 21s
320:	learn: -2.4032252	total: 3m 28s	remaining: 7m 20s
321:	learn: -2.4029622	total: 3m 29s	remaining: 7m 20s
322:	learn

455:	learn: -2.3686973	total: 4m 57s	remaining: 5m 54s
456:	learn: -2.3685303	total: 4m 58s	remaining: 5m 54s
457:	learn: -2.3681715	total: 4m 58s	remaining: 5m 53s
458:	learn: -2.3678351	total: 4m 59s	remaining: 5m 53s
459:	learn: -2.3676269	total: 5m	remaining: 5m 52s
460:	learn: -2.3673702	total: 5m	remaining: 5m 51s
461:	learn: -2.3671650	total: 5m 1s	remaining: 5m 51s
462:	learn: -2.3669710	total: 5m 2s	remaining: 5m 50s
463:	learn: -2.3665558	total: 5m 2s	remaining: 5m 49s
464:	learn: -2.3664341	total: 5m 3s	remaining: 5m 49s
465:	learn: -2.3661353	total: 5m 4s	remaining: 5m 48s
466:	learn: -2.3659400	total: 5m 4s	remaining: 5m 48s
467:	learn: -2.3656097	total: 5m 5s	remaining: 5m 47s
468:	learn: -2.3653950	total: 5m 6s	remaining: 5m 46s
469:	learn: -2.3649995	total: 5m 6s	remaining: 5m 46s
470:	learn: -2.3648161	total: 5m 7s	remaining: 5m 45s
471:	learn: -2.3647194	total: 5m 8s	remaining: 5m 44s
472:	learn: -2.3645026	total: 5m 8s	remaining: 5m 44s
473:	learn: -2.3642432	total: 

606:	learn: -2.3369247	total: 6m 38s	remaining: 4m 17s
607:	learn: -2.3367407	total: 6m 38s	remaining: 4m 17s
608:	learn: -2.3365985	total: 6m 39s	remaining: 4m 16s
609:	learn: -2.3363698	total: 6m 40s	remaining: 4m 15s
610:	learn: -2.3361908	total: 6m 41s	remaining: 4m 15s
611:	learn: -2.3359773	total: 6m 41s	remaining: 4m 14s
612:	learn: -2.3357496	total: 6m 42s	remaining: 4m 14s
613:	learn: -2.3355487	total: 6m 43s	remaining: 4m 13s
614:	learn: -2.3355204	total: 6m 43s	remaining: 4m 12s
615:	learn: -2.3353007	total: 6m 44s	remaining: 4m 12s
616:	learn: -2.3351562	total: 6m 45s	remaining: 4m 11s
617:	learn: -2.3350514	total: 6m 45s	remaining: 4m 10s
618:	learn: -2.3348211	total: 6m 46s	remaining: 4m 10s
619:	learn: -2.3344681	total: 6m 47s	remaining: 4m 9s
620:	learn: -2.3343295	total: 6m 48s	remaining: 4m 9s
621:	learn: -2.3339759	total: 6m 48s	remaining: 4m 8s
622:	learn: -2.3338593	total: 6m 49s	remaining: 4m 7s
623:	learn: -2.3335560	total: 6m 50s	remaining: 4m 7s
624:	learn: -2.

757:	learn: -2.3103962	total: 8m 19s	remaining: 2m 39s
758:	learn: -2.3102029	total: 8m 19s	remaining: 2m 38s
759:	learn: -2.3099989	total: 8m 20s	remaining: 2m 38s
760:	learn: -2.3098309	total: 8m 21s	remaining: 2m 37s
761:	learn: -2.3097621	total: 8m 21s	remaining: 2m 36s
762:	learn: -2.3094982	total: 8m 22s	remaining: 2m 36s
763:	learn: -2.3093357	total: 8m 23s	remaining: 2m 35s
764:	learn: -2.3092781	total: 8m 23s	remaining: 2m 34s
765:	learn: -2.3091616	total: 8m 24s	remaining: 2m 34s
766:	learn: -2.3091519	total: 8m 25s	remaining: 2m 33s
767:	learn: -2.3089290	total: 8m 25s	remaining: 2m 32s
768:	learn: -2.3089193	total: 8m 26s	remaining: 2m 32s
769:	learn: -2.3087239	total: 8m 27s	remaining: 2m 31s
770:	learn: -2.3085363	total: 8m 27s	remaining: 2m 30s
771:	learn: -2.3083488	total: 8m 28s	remaining: 2m 30s
772:	learn: -2.3081388	total: 8m 29s	remaining: 2m 29s
773:	learn: -2.3081370	total: 8m 29s	remaining: 2m 28s
774:	learn: -2.3078948	total: 8m 30s	remaining: 2m 28s
775:	learn

907:	learn: -2.2887898	total: 9m 58s	remaining: 1m
908:	learn: -2.2886205	total: 9m 58s	remaining: 59.9s
909:	learn: -2.2884389	total: 9m 59s	remaining: 59.3s
910:	learn: -2.2881262	total: 10m	remaining: 58.6s
911:	learn: -2.2881005	total: 10m	remaining: 58s
912:	learn: -2.2880393	total: 10m 1s	remaining: 57.3s
913:	learn: -2.2879147	total: 10m 2s	remaining: 56.7s
914:	learn: -2.2877368	total: 10m 2s	remaining: 56s
915:	learn: -2.2876048	total: 10m 3s	remaining: 55.3s
916:	learn: -2.2875069	total: 10m 4s	remaining: 54.7s
917:	learn: -2.2874843	total: 10m 4s	remaining: 54s
918:	learn: -2.2873082	total: 10m 5s	remaining: 53.4s
919:	learn: -2.2871419	total: 10m 6s	remaining: 52.7s
920:	learn: -2.2869455	total: 10m 6s	remaining: 52s
921:	learn: -2.2867125	total: 10m 7s	remaining: 51.4s
922:	learn: -2.2866240	total: 10m 8s	remaining: 50.7s
923:	learn: -2.2865773	total: 10m 8s	remaining: 50.1s
924:	learn: -2.2864699	total: 10m 9s	remaining: 49.4s
925:	learn: -2.2861743	total: 10m 10s	remaini

ValueError: y_true and y_pred contain different number of classes 36, 35. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35]

logistic

In [58]:
from sklearn.linear_model import LogisticRegression
model4 = LogisticRegression()
model4.fit(df_val,y_val)
y_final=model4.predict_proba(df_test)
print (log_loss(y_test,y_final));

C:\anaconda\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: y_true and y_pred contain different number of classes 36, 35. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35]

lightGBM

In [ ]:
model5= lgb.LGBMClassifier(objective='multiclass')

model5.fit(df_val,y_val)
y_final=model5.predict_proba(df_test)
print (log_loss(y_test,y_final));

XGBoost

In [59]:
model6= xgb.XGBClassifier(objective='multi:softprob')
model6.fit(X_train,y_train)
y_final=model6.predict_proba(X_test)
print (log_loss(y_test,y_final));

C:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


2.4607208844161397


# LogLoss


In [60]:
temp = data['Category']

le.fit_transform(temp)
le.classes_
y_final= pd.DataFrame(y_final, index=Id_test,columns  = le.classes_)
y_final.to_csv("submit.csv", float_format = '%.5F')


ValueError: Shape of passed values is (36, 33000), indices imply (36, 10000)